In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [4]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    a = np.array(a)
    b = np.array(b)
    c = np.array(b)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    if (angle > 180):
        return 360-angle;
    return angle

In [5]:
def frame_array(video, info):
    all_frames = []
    cap = cv2.VideoCapture(os.path.join('videos',video))
    fps = cap.get(cv2.CAP_PROP_FPS)
    dimensions = (int(cap.get(3)),int(cap.get(4)))
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        """try:
            frame = cv2.resize(frame, (200,200))
        except:
            break"""
        all_frames.append(frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Close down everything
    if info:
        dimensions = (int(cap.get(3)),int(cap.get(4)))
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        cv2.destroyAllWindows()
        return all_frames, fps, dimensions
    else:
        return all_frames

In [6]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle

In [7]:
# nose 0, left eye 1, right eye 2, left ear 3, right ear 4, 
# left shoulder 5, right shoulder 6, left elbow 7, right elbow 8, 
# left wrist 9, right wrist 10, left hip 11, right hip 12, left knee 13, right knee 14, left ankle 15, right ankle 16
# elbow, shoulder, hip, ankle

In [8]:
EDGES = {
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [9]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1-5]
        y2, x2, c2 = shaped[p2-5]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [10]:
def findAvgAnglesJJS(frame_array):
    l_one = [] # l = last
    l_l_one = []
    went_down = False
    jj_count = 0
    h_angle = [[0,0], [0,0]] # each row is left & rt, each column is up & down
    l_angle = [[0,0], [0,0]]
    for frame_idx, frame in enumerate(frame_array):
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)
        
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        kyps_w_scores = interpreter.get_tensor(output_details[0]['index'])

        # 5,6 shoulder; 7,8 elbow; 11,12 hip; 15,16 ankle
        wanted_scores = [kyps_w_scores[0][0][5], kyps_w_scores[0][0][6], kyps_w_scores[0][0][7], kyps_w_scores[0][0][8], 
                         kyps_w_scores[0][0][11], kyps_w_scores[0][0][12], kyps_w_scores[0][0][15], kyps_w_scores[0][0][16]]
        
        underarm_angle = [calculate_angle(wanted_scores[7], wanted_scores[5], wanted_scores[11])] + [calculate_angle(wanted_scores[8], wanted_scores[6], wanted_scores[12])]
        leg_angle = [calculate_angle(wanted_scores[5], wanted_scores[11], wanted_scores[15])] + [calculate_angle(wanted_scores[6], wanted_scores[12], wanted_scores[16])]
        
        if (frame_idx < 2):
            l_l_one = l_one
            l_one = [underarm_angle, leg_angle]
            continue
        
        if (went_down and (l_l_one[0][0] < l_one[0] < underarm_angle[0])):
            jj_count += 1
            print(jj_count)
        
        l_l_one = l_one
        l_one = [underarm_angle, leg_angle]
        
    h_avg_up = h_angle_up / jj_count
    h_avg_down = h_angle_down / jj_count
    l_avg_up = l_angle_up / jj_count
    l_avg_down = l_angle_down / jj_count
    return h_avg_up, h_avg_down, l_avg_up, l_avg_down

In [11]:
# workout
# play 10 second timer
# play workout video
# follow along
# show similarity score in a popup next to it
# save similarity score and live video
# point out mistakes and differences

def workout(instruct_vid, speed): #, output_name
    # cap1 = cv2.VideoCapture(os.path.join('videos','10secCountdown.mp4'))
    # cap2 = cv2.VideoCapture(os.path.join('videos','jumpingjacks.mp4'))
    all_frames_vid, fps, dimensions = frame_array(instruct_vid, True)
    
    # show live video at same time as workout video
    cap_live = cv2.VideoCapture(0)
    fps_live = cap_live.get(cv2.CAP_PROP_FPS)
    dimensions_live = (int(cap_live.get(3)),int(cap_live.get(4)))
    """
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap_live.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(os.path.join('videos','outputs',output_name), 
                                   fourcc, fps, (int(cap_live.get(3)),int(cap_live.get(4)))) """
    live_frames = [0] * len(all_frames_vid2)
    save_frame1 = []
    save_frame2 = []
    for frame_idx, frame2 in enumerate(all_frames_vid2):
        if (frame2 is None):
            break
        if cv2.waitKey(speed) & 0xFF == ord('q'):
            break
        cv2.imshow('Video Player', frame2)
        ret_live, frame_live = cap_live.read()
        live_frames[frame_idx] = frame_live
        if frame_idx == 50 or frame_idx == 100:
            save_frame1.append(frame2)
            save_frame2.append(frame_live)
        # video_writer.write(frame_live)    
    
    cap_live.release()
    cv2.destroyAllWindows()
    #return all_frames_vid2, live_frames, fps2, fps_live, dimensions2, dimensions_live
    return save_frame1, save_frame2

In [12]:
def test():
    l_one = [] # l = last
    l_l_one = []
    went_down = True
    jj_count = 0
    h_angle = [[0,0], [0,0]] # each row is left & rt, each column is up & down
    l_angle = [[0,0], [0,0]]
    frame_idx = 0
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)
        
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        kyps_w_scores = interpreter.get_tensor(output_details[0]['index'])

        # 5,6 shoulder; 7,8 elbow; 11,12 hip; 15,16 ankle
        wanted_scores = kyps_w_scores[0][0][5:]
        underarm_angle = [calculate_angle(wanted_scores[2], wanted_scores[0], wanted_scores[6])] + [calculate_angle(wanted_scores[3], wanted_scores[1], wanted_scores[7])]
        leg_angle = [calculate_angle(wanted_scores[0], wanted_scores[6], wanted_scores[10])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[11])]
        
        if (frame_idx < 2):
            l_l_one = l_one
            l_one = [underarm_angle, leg_angle]
            frame_idx += 1
            continue
            
        if (went_down and l_l_one[0][0] < l_one[0][0]  and l_one[0][0] > underarm_angle[0]):
            print(underarm_angle[0])
            jj_count += 1
            # print(jj_count)
        
        l_l_one = l_one
        l_one = [underarm_angle, leg_angle]
        cv2.putText(frame,  
                str(underarm_angle[0]),  
                (50, 50),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 255, 255),  
                2,  
                cv2.LINE_4)
        draw_keypoints(frame, wanted_scores, 0.5)
        draw_connections(frame, wanted_scores, EDGES, 0.5)
        cv2.imshow('Feed', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    h_avg_up = h_angle_up / jj_count
    h_avg_down = h_angle_down / jj_count
    l_avg_up = l_angle_up / jj_count
    l_avg_down = l_angle_down / jj_count
    return h_avg_up, h_avg_down, l_avg_up, l_avg_down

In [12]:
test()

80.77784010599193
75.1119796234129
74.72528846355264
80.12936145562341
42.54599624572354
42.46721342874203
42.30372602076861
71.45797868161783
21.34112413719077
22.654467061746672
23.929940002449438
25.255345041079583
25.69817352572819
25.603030698137204
25.03396494970113
25.454998300708226
25.495402284743438
25.77763594653212
25.62470630346824
26.833253635588637
26.619745337228622
25.176386346726943
27.074721313316097
27.08036475711782
26.79929735012992
26.914860735791947
24.862469144925864
26.646179876863865
26.135413208088433
25.251381823813684
25.063833366941594
24.55784441549059
96.06036105313878
96.71176985466124
101.88426528292273
102.53947849981287
101.54706567378837
101.23830697235793
93.7171280545825
89.49103729809633
88.69350342947062
85.15655143948204
115.42452791620438
116.19142155621341
111.32215237615465
87.53063640251926
92.44715683125618
92.90643924161536
67.36994653882512
63.47355584376463
51.2973981009504
68.67576942510483
67.83763691205257
66.81295144219975
68.04541

NameError: name 'h_angle_up' is not defined